In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests
import json
from datetime import *


StatementMeta(, e84ec6e3-2a3e-4716-bfe8-5833c10c734e, 3, Finished, Available, Finished)

In [2]:
# Variáveis
moeda = 'USD'

# data_inicial = '01-01-2024'
# data_final = '12-31-2025'
data_inicial = datetime.now().strftime('%m-%d-%Y') 
data_final = datetime.now().strftime('%m-%d-%Y') 

skip = 0
top = 100

StatementMeta(, e84ec6e3-2a3e-4716-bfe8-5833c10c734e, 4, Finished, Available, Finished)

In [3]:
# Lista de moedas
df_moedas = spark.sql("SELECT DISTINCT Moeda FROM dim_moedas")
lista_moedas = [row['Moeda'] for row in df_moedas.collect()]
print(lista_moedas)

StatementMeta(, e84ec6e3-2a3e-4716-bfe8-5833c10c734e, 5, Finished, Available, Finished)

['DKK', 'GBP', 'CHF', 'CAD', 'EUR', 'NOK', 'AUD', 'JPY', 'USD', 'SEK']


In [4]:
# Iteração por cada moeda
for moeda in lista_moedas:

    skip = 0

    df_total = []

    while True:

        url = (
            f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
            f"CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?"
            f"@moeda='{moeda}'&@dataInicial='{data_inicial}'&@dataFinalCotacao='{data_final}'&"
            f"$top={top}&$skip={skip}&$filter=tipoBoletim%20eq%20'Fechamento'&$format=json&"
            f"$select=cotacaoCompra,dataHoraCotacao"
        )

        response = requests.get(url)

        dados = response.json()['value']

        if not dados: 
            break
    
        df_total.extend(dados)

        skip += top


    data_inicial_path = datetime.strptime(data_inicial, "%m-%d-%Y").strftime("%Y%m%d")
    data_final_path = datetime.strptime(data_final, "%m-%d-%Y").strftime("%Y%m%d")

    path = (
        f"Files/Cotacoes/Novos/"
        f"{moeda}-"
        f"{data_inicial_path}_"
        f"{data_final_path}"
        f".parquet"
    )

    if df_total:
        df = spark.createDataFrame(df_total) \
            .withColumn('moeda', lit(moeda)) \

        df.write.mode('overwrite').parquet(path)

StatementMeta(, e84ec6e3-2a3e-4716-bfe8-5833c10c734e, 6, Finished, Available, Finished)